In [1]:
# loading libraries

import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
# creating a temp df (with duplicated lines for airports that changed their ICAO code) to add clusters column to map df and flights df

In [4]:
temp = pd.read_csv("./assets/csv/temp/temp_icao_cluster.csv", sep=",")

In [5]:
temp

,icao_code,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,-23.006944,-47.134444,Campinas,SP,3
4,SBGL,-22.810000,-43.250556,Rio de janeiro,RJ,2
...,...,...,...,...,...,...
237,SSLT,-29.812500,-55.893333,Alegrete,RS,8
238,SILC,-13.037778,-55.950278,Lucas do rio verde,MT,8
239,SIZX,-11.296667,-57.548889,Juara,MT,8
240,SSOE,-26.781111,-53.503333,São miguel do oeste,SC,8


# Cleaning *'clusters' and 'flights'* files from 2012 to 2021

In [6]:
# 2012 clusters – Arealva (SBAE) repeated

In [7]:
clusters_2012 = pd.read_csv('./assets/csv/count/df2012.csv', sep=',')

In [8]:
clusters_2012

,icao_code,count
0,SBKP,46337
1,SBSP,38886
2,SBBR,37624
3,SBGR,34239
4,SBGL,31249
...,...,...
84,SBCP,99
85,SBDO,57
86,SJTC,26
87,SSZR,24


In [9]:
clusters_2012 = clusters_2012.merge(temp, how='left', on="icao_code")

In [10]:
clusters_2012

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBKP,46337,-23.006944,-47.134444,Campinas,SP,3
1,SBSP,38886,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,37624,-15.871111,-47.918611,Brasília,DF,1
3,SBGR,34239,-23.435556,-46.473056,Guarulhos,SP,1
4,SBGL,31249,-22.810000,-43.250556,Rio de janeiro,RJ,2
...,...,...,...,...,...,...,...
84,SBCP,99,-21.701111,-41.307778,Campos dos goytacazes,RJ,8
85,SBDO,57,-22.200556,-54.925556,Dourados,MS,8
86,SJTC,26,-22.157778,-49.068333,Arealva,SP,8
87,SSZR,24,-27.908889,-54.522222,Santa rosa,RS,8


In [11]:
clusters_2012.city_primary.duplicated().any()

True

In [12]:
clusters_2012[clusters_2012.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBRJ,23963,-22.910000,-43.162500,Rio de janeiro,RJ,3
86,SJTC,26,-22.157778,-49.068333,Arealva,SP,8


In [13]:
clusters_2012[clusters_2012['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
4,SBGL,31249,-22.81,-43.250556,Rio de janeiro,RJ,2
6,SBRJ,23963,-22.91,-43.162500,Rio de janeiro,RJ,3


In [14]:
clusters_2012[clusters_2012['city_primary'] == 'Arealva']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
46,SBAE,997,-22.157778,-49.068333,Arealva,SP,8
86,SJTC,26,-22.157778,-49.068333,Arealva,SP,8


In [15]:
clusters_2012.at[46, 'count'] = 997 + 26

In [16]:
clusters_2012.drop(86, inplace=True)

In [17]:
clusters_2012 = clusters_2012.reset_index(drop=True)
clusters_2012

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBKP,46337,-23.006944,-47.134444,Campinas,SP,3
1,SBSP,38886,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,37624,-15.871111,-47.918611,Brasília,DF,1
3,SBGR,34239,-23.435556,-46.473056,Guarulhos,SP,1
4,SBGL,31249,-22.810000,-43.250556,Rio de janeiro,RJ,2
...,...,...,...,...,...,...,...
83,SBCN,106,-17.724722,-48.610000,Caldas novas,GO,8
84,SBCP,99,-21.701111,-41.307778,Campos dos goytacazes,RJ,8
85,SBDO,57,-22.200556,-54.925556,Dourados,MS,8
86,SSZR,24,-27.908889,-54.522222,Santa rosa,RS,8


In [18]:
clusters_2012.to_csv("./assets/csv/cluster/cluster_2012_map.csv", sep=",")

In [ ]:
# flights 2012

In [24]:
flights_2012 = pd.read_csv("./assets/csv/temp/flights_2012.csv", sep=",", index_col=0).drop(columns="index")

In [25]:
len(flights_2012)

474174

In [26]:
len(flights_2012[flights_2012.icao_origin == flights_2012.icao_dest])

0

In [27]:
flights_2012.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,AZU,AD,4461,2,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2012-08-18,07:12:00,2012-08-18,2012,8,18,6,AGOSTO,SBPA,POA,Salgado Filho,Porto Alegre,RS,SUL,08:17:00,2012-08-18,2012,8,18,E195,EMBRAER 195,1579.0,15900,534.0,1.08,494.04,118,48,0,978,0,0,0,0,4578,0.0,0,0,0,522252,0,63012,25632,8490,8490,4461_2,SBCT-SBPA,CWB-POA
1,AZU,AD,4092,3,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2012-08-01,14:43:00,2012-08-01,2012,8,1,3,AGOSTO,SBMG,MGF,Sílvio Name Júnior,Maringá,PR,SUL,15:44:00,2012-08-01,2012,8,1,E195,EMBRAER 195,1230.0,17965,367.0,1.02,359.42,118,47,1,953,0,28,0,0,4581,367.0,10276,0,0,349751,0,43306,17249,6593,6593,4092_3,SBCT-SBMG,CWB-MGF


In [30]:
flights_2012.loc[flights_2012.icao_origin == "SJTC", "icao_origin"] = "SBAE"

In [31]:
len(flights_2012[flights_2012.sched_date.isnull()])

0

In [32]:
len(flights_2012[flights_2012.dep_date.isnull()])

0

In [33]:
flights_2012.insert(11, 'sched_year', flights_2012.sched_date.str[0:4])
flights_2012.insert(12, 'sched_month', flights_2012.sched_date.str[5:7])
flights_2012.insert(13, 'sched_day', flights_2012.sched_date.str[8:10])

In [34]:
sorted(flights_2012.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [35]:
# creating cluster column for origin airport

In [36]:
flights_2012 = flights_2012.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [37]:
len(flights_2012[flights_2012.cluster.isnull()])

0

In [38]:
flights_2012 = flights_2012.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [39]:
sorted(flights_2012.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [40]:
# creating cluster column for destination airport

In [41]:
flights_2012 = flights_2012.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [42]:
len(flights_2012[flights_2012.cluster.isnull()])

0

In [43]:
flights_2012 = flights_2012.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [44]:
sorted(flights_2012.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [45]:
flights_2012.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,sched_year,sched_month,sched_day,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata,lat_dd_origin,long_dd_origin,cluster_origin,lat_dd_dest,long_dd_dest,cluster_dest
0,AZU,AD,4461,2,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2012-08-18,2012,08,18,07:12:00,2012-08-18,2012,8,18,6,AGOSTO,SBPA,POA,Salgado Filho,Porto Alegre,RS,SUL,08:17:00,2012-08-18,2012,8,18,E195,EMBRAER 195,1579.0,15900,534.0,1.08,494.04,118,48,0,978,0,0,0,0,4578,0.0,0,0,0,522252,0,63012,25632,8490,8490,4461_2,SBCT-SBPA,CWB-POA,-25.531667,-49.176111,4,-29.994722,-51.171111,3
1,AZU,AD,4092,3,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2012-08-01,2012,08,01,14:43:00,2012-08-01,2012,8,1,3,AGOSTO,SBMG,MGF,Sílvio Name Júnior,Maringá,PR,SUL,15:44:00,2012-08-01,2012,8,1,E195,EMBRAER 195,1230.0,17965,367.0,1.02,359.42,118,47,1,953,0,28,0,0,4581,367.0,10276,0,0,349751,0,43306,17249,6593,6593,4092_3,SBCT-SBMG,CWB-MGF,-25.531667,-49.176111,4,-23.479444,-52.012222,6


In [47]:
# 2013 clusters – São gonçalo do amarante (SBSG) repeated

In [48]:
clusters_2013 = pd.read_csv('./assets/csv/count/df2013.csv', sep=',')

In [49]:
clusters_2013

,icao_code,count
0,SBGR,55271
1,SBKP,53930
2,SBBR,46896
3,SBSP,44278
4,SBCF,39197
...,...,...
127,SBTG,8
128,SBLJ,4
129,SSOE,3
130,SSCK,2


In [50]:
clusters_2013 = clusters_2013.merge(temp, how='left', on="icao_code")

In [51]:
clusters_2013

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,55271,-23.435556,-46.473056,Guarulhos,SP,1
1,SBKP,53930,-23.006944,-47.134444,Campinas,SP,3
2,SBBR,46896,-15.871111,-47.918611,Brasília,DF,1
3,SBSP,44278,-23.626111,-46.656389,São paulo,SP,2
4,SBCF,39197,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
127,SBTG,8,-20.751389,-51.680278,Três lagoas,MS,8
128,SBLJ,4,-27.782222,-50.281667,Lages,SC,8
129,SSOE,3,-26.781111,-53.503333,São miguel do oeste,SC,8
130,SSCK,2,-27.180556,-52.051111,Concórdia,SC,8


In [52]:
clusters_2013.city_primary.duplicated().any()

True

In [53]:
clusters_2013[clusters_2013.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,31326,-22.810000,-43.250556,Rio de janeiro,RJ,2
121,SBSG,35,-5.768889,-35.366389,São gonçalo do amarante,RN,4


In [54]:
clusters_2013[clusters_2013['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
5,SBRJ,36008,-22.91,-43.162500,Rio de janeiro,RJ,3
6,SBGL,31326,-22.81,-43.250556,Rio de janeiro,RJ,2


In [55]:
clusters_2013[clusters_2013['city_primary'] == 'São gonçalo do amarante']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
18,SBNT,5996,-5.768889,-35.366389,São gonçalo do amarante,RN,4
121,SBSG,35,-5.768889,-35.366389,São gonçalo do amarante,RN,4


In [56]:
clusters_2013.at[121, 'count'] = 5996 + 35

In [57]:
clusters_2013.drop(18, inplace=True)

In [58]:
clusters_2013 = clusters_2013.reset_index(drop=True)
clusters_2013

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,55271,-23.435556,-46.473056,Guarulhos,SP,1
1,SBKP,53930,-23.006944,-47.134444,Campinas,SP,3
2,SBBR,46896,-15.871111,-47.918611,Brasília,DF,1
3,SBSP,44278,-23.626111,-46.656389,São paulo,SP,2
4,SBCF,39197,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
126,SBTG,8,-20.751389,-51.680278,Três lagoas,MS,8
127,SBLJ,4,-27.782222,-50.281667,Lages,SC,8
128,SSOE,3,-26.781111,-53.503333,São miguel do oeste,SC,8
129,SSCK,2,-27.180556,-52.051111,Concórdia,SC,8


In [59]:
clusters_2013.to_csv("./assets/csv/cluster/cluster_2013_map.csv", sep=",")

In [ ]:
# flights 2013

In [60]:
flights_2013 = pd.read_csv("./assets/csv/temp/flights_2013.csv", sep=",", index_col=0).drop(columns="index")

In [61]:
len(flights_2013)

640991

In [62]:
len(flights_2013[flights_2013.icao_origin == flights_2013.icao_dest])

1

In [63]:
flights_2013 = flights_2013[~(flights_2013.icao_origin == flights_2013.icao_dest)]

In [64]:
flights_2013.reset_index(drop=True, inplace=True)

In [65]:
len(flights_2013)

640990

In [66]:
flights_2013.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,AZU,AD,4027,1,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2013-05-01,20:49:00,2013-05-01,2013,5,1,3,MAIO,SBKP,VCP,Viracopos,Campinas,SP,SUDESTE,21:51:00,2013-05-01,2013,5,1,E190,EMBRAER 190,2129.0,15982,349.0,1.03,338.60,106,86,1,725,76,0,0,0,7326,349.0,0,0,0,253025,26524,36994,30014,5577,5577,4027_1,SBCT-SBKP,CWB-VCP
1,AZU,AD,6956,1,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2013-05-01,08:48:00,2013-05-01,2013,5,1,3,MAIO,SBGR,GRU,Guarulhos - Governador André Franco Montoro,Guarulhos,SP,SUDESTE,09:58:00,2013-05-01,2013,5,1,E195,EMBRAER 195,2439.0,15841,359.0,1.17,307.14,118,93,3,1027,135,0,0,0,8362,1077.0,0,0,0,368693,48465,42362,33387,5686,5686,6956_1,SBCT-SBGR,CWB-GRU


In [69]:
flights_2013.loc[flights_2013.icao_origin == "SBNT", "icao_origin"] = "SBSG"

In [70]:
len(flights_2013[flights_2013.sched_date.isnull()])

0

In [71]:
len(flights_2013[flights_2013.dep_date.isnull()])

0

In [72]:
flights_2013.insert(11, 'sched_year', flights_2013.sched_date.str[0:4])
flights_2013.insert(12, 'sched_month', flights_2013.sched_date.str[5:7])
flights_2013.insert(13, 'sched_day', flights_2013.sched_date.str[8:10])

In [73]:
sorted(flights_2013.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [74]:
# creating cluster column for origin airport

In [75]:
flights_2013 = flights_2013.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [76]:
len(flights_2013[flights_2013.cluster.isnull()])

0

In [77]:
flights_2013 = flights_2013.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [78]:
sorted(flights_2013.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [79]:
# creating cluster column for destination airport

In [80]:
flights_2013 = flights_2013.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [81]:
len(flights_2013[flights_2013.cluster.isnull()])

0

In [82]:
flights_2013 = flights_2013.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [83]:
sorted(flights_2013.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [84]:
flights_2013.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,sched_year,sched_month,sched_day,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata,lat_dd_origin,long_dd_origin,cluster_origin,lat_dd_dest,long_dd_dest,cluster_dest
0,AZU,AD,4027,1,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2013-05-01,2013,05,01,20:49:00,2013-05-01,2013,5,1,3,MAIO,SBKP,VCP,Viracopos,Campinas,SP,SUDESTE,21:51:00,2013-05-01,2013,5,1,E190,EMBRAER 190,2129.0,15982,349.0,1.03,338.60,106,86,1,725,76,0,0,0,7326,349.0,0,0,0,253025,26524,36994,30014,5577,5577,4027_1,SBCT-SBKP,CWB-VCP,-25.531667,-49.176111,4,-23.006944,-47.134444,3
1,AZU,AD,6956,1,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2013-05-01,2013,05,01,08:48:00,2013-05-01,2013,5,1,3,MAIO,SBGR,GRU,Guarulhos - Governador André Franco Montoro,Guarulhos,SP,SUDESTE,09:58:00,2013-05-01,2013,5,1,E195,EMBRAER 195,2439.0,15841,359.0,1.17,307.14,118,93,3,1027,135,0,0,0,8362,1077.0,0,0,0,368693,48465,42362,33387,5686,5686,6956_1,SBCT-SBGR,CWB-GRU,-25.531667,-49.176111,4,-23.435556,-46.473056,1


In [85]:
flights_2013.to_csv('./assets/csv/flights_2013.csv', sep=',')

In [168]:
# 2014 clusters – São gonçalo do amarante (SBSG) repeated

In [86]:
clusters_2014 = pd.read_csv('./assets/csv/count/df2014.csv', sep=',')

In [87]:
clusters_2014

,icao_code,count
0,SBGR,65891
1,SBBR,59198
2,SBKP,56746
3,SBSP,51452
4,SBRJ,43042
...,...,...
115,SWVC,20
116,SJHG,19
117,SBVG,11
118,SWCA,6


In [88]:
clusters_2014 = clusters_2014.merge(temp, how='left', on="icao_code")

In [89]:
clusters_2014

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,65891,-23.435556,-46.473056,Guarulhos,SP,1
1,SBBR,59198,-15.871111,-47.918611,Brasília,DF,1
2,SBKP,56746,-23.006944,-47.134444,Campinas,SP,3
3,SBSP,51452,-23.626111,-46.656389,São paulo,SP,2
4,SBRJ,43042,-22.910000,-43.162500,Rio de janeiro,RJ,3
...,...,...,...,...,...,...,...
115,SWVC,20,-9.981111,-51.140556,Vila rica,MT,8
116,SJHG,19,-10.633296,-51.565836,Confresa,MT,8
117,SBVG,11,-21.588889,-45.473333,Varginha,MG,8
118,SWCA,6,-4.871389,-66.897500,Carauari,AM,8


In [90]:
clusters_2014.city_primary.duplicated().any()

True

In [91]:
clusters_2014[clusters_2014.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,31525,-22.810000,-43.250556,Rio de janeiro,RJ,2
47,SBNT,2027,-5.768889,-35.366389,São gonçalo do amarante,RN,4


In [92]:
clusters_2014[clusters_2014['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
4,SBRJ,43042,-22.91,-43.162500,Rio de janeiro,RJ,3
6,SBGL,31525,-22.81,-43.250556,Rio de janeiro,RJ,2


In [93]:
clusters_2014[clusters_2014['city_primary'] == 'São gonçalo do amarante']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
25,SBSG,5074,-5.768889,-35.366389,São gonçalo do amarante,RN,4
47,SBNT,2027,-5.768889,-35.366389,São gonçalo do amarante,RN,4


In [94]:
clusters_2014.at[25, 'count'] = 5074 + 2027

In [95]:
clusters_2014.drop(47, inplace=True)

In [96]:
clusters_2014 = clusters_2014.reset_index(drop=True)
clusters_2014

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,65891,-23.435556,-46.473056,Guarulhos,SP,1
1,SBBR,59198,-15.871111,-47.918611,Brasília,DF,1
2,SBKP,56746,-23.006944,-47.134444,Campinas,SP,3
3,SBSP,51452,-23.626111,-46.656389,São paulo,SP,2
4,SBRJ,43042,-22.910000,-43.162500,Rio de janeiro,RJ,3
...,...,...,...,...,...,...,...
114,SWVC,20,-9.981111,-51.140556,Vila rica,MT,8
115,SJHG,19,-10.633296,-51.565836,Confresa,MT,8
116,SBVG,11,-21.588889,-45.473333,Varginha,MG,8
117,SWCA,6,-4.871389,-66.897500,Carauari,AM,8


In [97]:
clusters_2014.to_csv("./assets/csv/cluster/cluster_2014_map.csv", sep=",")

In [98]:
# flights 2014

In [104]:
flights_2014 = pd.read_csv("./assets/csv/temp/flights_2014.csv", sep=",", index_col=0).drop(columns="index")

In [105]:
len(flights_2014)

719419

In [106]:
len(flights_2014[flights_2014.icao_origin == flights_2014.icao_dest])

0

In [107]:
flights_2014.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,ONE,O6,6214,1,SBGL,GIG,Aeroporto Internacional Do Rio De Janeiro (Gal...,Rio De Janeiro,RJ,4,2014-09-01,18:04:00,2014-09-01,2014,9,1,1,SETEMBRO,SBBR,BSB,Presidente Juscelino Kubitschek,Brasília,DF,CENTRO-OESTE,20:06:00,2014-09-01,2014,9,1,A318,AIRBUS A318,4733.0,15330,914.0,2.03,450.23,120,102,0,765,0,0,0,0,8415,0.0,0,0,0,699210,0,109680,93228,14011,14011,6214_1,SBGL-SBBR,GIG-BSB
1,ONE,O6,6317,1,SBRF,REC,Guararapes - Gilberto Freyre,Recife,PE,2,2014-09-01,06:40:00,2014-09-01,2014,9,1,1,SETEMBRO,SBGR,GRU,Guarulhos - Governador André Franco Montoro,Guarulhos,SP,SUDESTE,10:29:00,2014-09-01,2014,9,1,A320,AIRBUS A320-100/200,9317.0,18500,2101.0,3.82,549.97,162,139,1,1723,38,0,0,0,12261,2101.0,0,0,0,3620020,79838,340362,292039,38868,38868,6317_1,SBRF-SBGR,REC-GRU


In [108]:
flights_2014.loc[flights_2014.icao_origin == "SBNT", "icao_origin"] = "SBSG"

In [109]:
len(flights_2014[flights_2014.sched_date.isnull()])

0

In [110]:
len(flights_2014[flights_2014.dep_date.isnull()])

0

In [111]:
flights_2014.insert(11, 'sched_year', flights_2014.sched_date.str[0:4])
flights_2014.insert(12, 'sched_month', flights_2014.sched_date.str[5:7])
flights_2014.insert(13, 'sched_day', flights_2014.sched_date.str[8:10])

In [112]:
sorted(flights_2014.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [113]:
# creating cluster column for origin airport

In [114]:
flights_2014 = flights_2014.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [115]:
len(flights_2014[flights_2014.cluster.isnull()])

0

In [116]:
flights_2014 = flights_2014.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [117]:
sorted(flights_2014.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [118]:
# creating cluster column for destination airport

In [119]:
flights_2014 = flights_2014.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [120]:
len(flights_2014[flights_2014.cluster.isnull()])

0

In [121]:
flights_2014 = flights_2014.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [122]:
sorted(flights_2014.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [124]:
# 2015 clusters – Rondonópolis (SBRD) repeated

In [125]:
clusters_2015 = pd.read_csv('./assets/csv/count/df2015.csv', sep=',')

In [126]:
clusters_2015

,icao_code,count
0,SBGR,65369
1,SBBR,62006
2,SBSP,56075
3,SBKP,53923
4,SBCF,45028
...,...,...
113,SWKO,53
114,SBUG,51
115,SNVB,43
116,SNDV,34


In [127]:
clusters_2015 = clusters_2015.merge(temp, how='left', on="icao_code")

In [128]:
clusters_2015

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,65369,-23.435556,-46.473056,Guarulhos,SP,1
1,SBBR,62006,-15.871111,-47.918611,Brasília,DF,1
2,SBSP,56075,-23.626111,-46.656389,São paulo,SP,2
3,SBKP,53923,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,45028,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
113,SWKO,53,-4.133889,-63.131111,Coari,AM,8
114,SBUG,51,-29.783333,-57.036944,Uruguaiana,RS,8
115,SNVB,43,-13.296389,-38.992500,Valença,BA,8
116,SNDV,34,-20.181944,-44.870000,Divinópolis,MG,8


In [129]:
clusters_2015.city_primary.duplicated().any()

True

In [130]:
clusters_2015[clusters_2015.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,32767,-22.810000,-43.250556,Rio de janeiro,RJ,2
80,SBRD,399,-16.588056,-54.721667,Rondonópolis,MT,8


In [131]:
clusters_2015[clusters_2015['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
5,SBRJ,42224,-22.91,-43.162500,Rio de janeiro,RJ,3
6,SBGL,32767,-22.81,-43.250556,Rio de janeiro,RJ,2


In [132]:
clusters_2015[clusters_2015['city_primary'] == 'Rondonópolis']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
76,SWRD,439,-16.588056,-54.721667,Rondonópolis,MT,8
80,SBRD,399,-16.588056,-54.721667,Rondonópolis,MT,8


In [133]:
clusters_2015.at[80, 'count'] = 439 + 399

In [134]:
clusters_2015.drop(76, inplace=True)

In [135]:
clusters_2015 = clusters_2015.reset_index(drop=True)
clusters_2015

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,65369,-23.435556,-46.473056,Guarulhos,SP,1
1,SBBR,62006,-15.871111,-47.918611,Brasília,DF,1
2,SBSP,56075,-23.626111,-46.656389,São paulo,SP,2
3,SBKP,53923,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,45028,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
112,SWKO,53,-4.133889,-63.131111,Coari,AM,8
113,SBUG,51,-29.783333,-57.036944,Uruguaiana,RS,8
114,SNVB,43,-13.296389,-38.992500,Valença,BA,8
115,SNDV,34,-20.181944,-44.870000,Divinópolis,MG,8


In [137]:
# flights 2015

In [138]:
flights_2015 = pd.read_csv("./assets/csv/temp/flights_2015.csv", sep=",", index_col=0).drop(columns="index")

In [139]:
len(flights_2015)

728814

In [140]:
len(flights_2015[flights_2015.icao_origin == flights_2015.icao_dest])

2

In [141]:
flights_2015 = flights_2015[~(flights_2015.icao_origin == flights_2015.icao_dest)]

In [142]:
flights_2015.reset_index(drop=True, inplace=True)

In [143]:
len(flights_2015)

728812

In [144]:
flights_2015.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,ONE,O6,6235,1,SBBR,BSB,Presidente Juscelino Kubitschek,Brasília,DF,3,2015-07-01,14:04:00,2015-07-01,2015,7,1,3,JULHO,SBRJ,SDU,Santos Dumont,Rio De Janeiro,RJ,SUDESTE,15:43:00,2015-07-01,2015,7,1,A318,AIRBUS A318,4101.0,15330,928.0,1.65,562.56,120,116,1,1020,2,140,0,0,9937,928.0,129920,0,0,946560,1856,111360,107648,14226,14226,6235_1,SBBR-SBRJ,BSB-SDU
1,ONE,O6,6291,1,SBBR,BSB,Presidente Juscelino Kubitschek,Brasília,DF,3,2015-07-01,09:27:00,2015-07-01,2015,7,1,3,JULHO,SBGO,GYN,Santa Genoveva/Goiânia,Goiânia,GO,CENTRO-OESTE,10:14:00,2015-07-01,2015,7,1,F100,FOKKER 100,1273.0,10454,163.0,0.78,208.59,100,55,0,672,0,1,0,0,4798,0.0,163,0,0,109536,0,16300,8965,1704,1704,6291_1,SBBR-SBGO,BSB-GYN


In [145]:
flights_2015.loc[flights_2015.icao_origin == "SWRD", "icao_origin"] = "SBRD"

In [146]:
len(flights_2015[flights_2015.sched_date.isnull()])

0

In [147]:
len(flights_2015[flights_2015.dep_date.isnull()])

0

In [148]:
flights_2015.insert(11, 'sched_year', flights_2015.sched_date.str[0:4])
flights_2015.insert(12, 'sched_month', flights_2015.sched_date.str[5:7])
flights_2015.insert(13, 'sched_day', flights_2015.sched_date.str[8:10])

In [149]:
sorted(flights_2015.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [150]:
# creating cluster column for origin airport

In [151]:
flights_2015 = flights_2015.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [152]:
len(flights_2015[flights_2015.cluster.isnull()])

0

In [153]:
flights_2015 = flights_2015.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [154]:
sorted(flights_2015.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [155]:
# creating cluster column for destination airport

In [156]:
flights_2015 = flights_2015.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [157]:
len(flights_2015[flights_2015.cluster.isnull()])

0

In [158]:
flights_2015 = flights_2015.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [159]:
sorted(flights_2015.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [161]:
# 2016 clusters 

In [162]:
clusters_2016 = pd.read_csv('./assets/csv/count/df2016.csv', sep=',')

In [163]:
clusters_2016

,icao_code,count
0,SBGR,58711
1,SBSP,57075
2,SBBR,52620
3,SBKP,46875
4,SBCF,38140
...,...,...
108,SBBW,40
109,SBPB,27
110,SNPD,7
111,SNDV,4


In [164]:
clusters_2016 = clusters_2016.merge(temp, how='left', on="icao_code")

In [165]:
clusters_2016

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,58711,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,57075,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,52620,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,46875,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,38140,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
108,SBBW,40,-15.860833,-52.389444,Barra do garças,MT,8
109,SBPB,27,-2.893889,-41.730278,Parnaíba,PI,8
110,SNPD,7,-18.672222,-46.491389,Patos de minas,MG,8
111,SNDV,4,-20.181944,-44.870000,Divinópolis,MG,8


In [166]:
clusters_2016.city_primary.duplicated().any()

True

In [167]:
clusters_2016[clusters_2016.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,31978,-22.81,-43.250556,Rio de janeiro,RJ,2


In [168]:
clusters_2016[clusters_2016['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
5,SBRJ,36964,-22.91,-43.162500,Rio de janeiro,RJ,3
6,SBGL,31978,-22.81,-43.250556,Rio de janeiro,RJ,2


In [170]:
# flights 2016

In [171]:
flights_2016 = pd.read_csv("./assets/csv/temp/flights_2016.csv", sep=",", index_col=0).drop(columns="index")

In [172]:
len(flights_2016)

632201

In [173]:
len(flights_2016[flights_2016.icao_origin == flights_2016.icao_dest])

0

In [174]:
flights_2016.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,ONE,O6,6004,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2016-09-03,09:40:00,2016-09-03,2016,9,3,6,SETEMBRO,SBRJ,SDU,Santos Dumont,Rio De Janeiro,RJ,SUDESTE,10:34:00,2016-09-03,2016,9,3,A319,AIRBUS A319,2525.0,16045,366.0,0.90,406.26,132,111,1,639,42,0,0,0,9081,366.0,0,0,0,233874,15372,48312,40626,5872,5872,6004_1,SBSP-SBRJ,CGH-SDU
1,ONE,O6,6033,1,SBRJ,SDU,Santos Dumont,Rio De Janeiro,RJ,4,2016-09-03,07:06:00,2016-09-03,2016,9,3,6,SETEMBRO,SBSP,CGH,Congonhas,São Paulo,SP,SUDESTE,08:15:00,2016-09-03,2016,9,3,A319,AIRBUS A319,2757.0,16038,366.0,1.15,317.94,132,59,0,162,0,0,0,0,4587,0.0,0,0,0,59292,0,48312,21594,5869,5869,6033_1,SBRJ-SBSP,SDU-CGH


In [175]:
len(flights_2016[flights_2016.sched_date.isnull()])

0

In [176]:
len(flights_2016[flights_2016.dep_date.isnull()])

0

In [177]:
flights_2016.insert(11, 'sched_year', flights_2016.sched_date.str[0:4])
flights_2016.insert(12, 'sched_month', flights_2016.sched_date.str[5:7])
flights_2016.insert(13, 'sched_day', flights_2016.sched_date.str[8:10])

In [178]:
sorted(flights_2016.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [179]:
# creating cluster column for origin airport

In [180]:
flights_2016 = flights_2016.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [181]:
len(flights_2016[flights_2016.cluster.isnull()])

0

In [182]:
flights_2016 = flights_2016.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [183]:
sorted(flights_2016.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [184]:
# creating cluster column for destination airport

In [185]:
flights_2016 = flights_2016.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [186]:
len(flights_2016[flights_2016.cluster.isnull()])

0

In [187]:
flights_2016 = flights_2016.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [188]:
sorted(flights_2016.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [190]:
# 2017 clusters 

In [191]:
clusters_2017 = pd.read_csv('./assets/csv/count/df2017.csv', sep=',')

In [192]:
clusters_2017

,icao_code,count
0,SBGR,59063
1,SBSP,57813
2,SBBR,46990
3,SBKP,43609
4,SBCF,36747
...,...,...
106,SBAX,54
107,SBPB,50
108,SNVB,48
109,SBFE,41


In [193]:
clusters_2017 = clusters_2017.merge(temp, how='left', on="icao_code")

In [194]:
clusters_2017

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,59063,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,57813,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,46990,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,43609,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,36747,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
106,SBAX,54,-19.560556,-46.965556,Araxá,MG,8
107,SBPB,50,-2.893889,-41.730278,Parnaíba,PI,8
108,SNVB,48,-13.296389,-38.992500,Valença,BA,8
109,SBFE,41,-12.200300,-38.906799,Feira de Santana,BA,8


In [195]:
clusters_2017.city_primary.duplicated().any()

True

In [196]:
clusters_2017[clusters_2017.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,32583,-22.81,-43.250556,Rio de janeiro,RJ,2


In [197]:
clusters_2017[clusters_2017['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
5,SBRJ,34357,-22.91,-43.162500,Rio de janeiro,RJ,3
6,SBGL,32583,-22.81,-43.250556,Rio de janeiro,RJ,2


In [199]:
# flights 2017

In [200]:
flights_2017 = pd.read_csv("./assets/csv/temp/flights_2017.csv", sep=",", index_col=0).drop(columns="index")

In [201]:
len(flights_2017)

612777

In [202]:
len(flights_2017[flights_2017.icao_origin == flights_2017.icao_dest])

0

In [203]:
flights_2017.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,AZU,AD,4285,1,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2017-02-03,09:20:00,2017-02-03,2017,2,3,5,FEVEREIRO,SBKP,VCP,Viracopos,Campinas,SP,SUDESTE,10:21:00,2017-02-03,2017,2,3,E195,EMBRAER 195,2057.0,12687,349.0,1.02,341.92,118,70,6,666,21,0,0,0,6387,2094.0,0,0,0,232434,7329,41182,24430,4427,4427,4285_1,SBCT-SBKP,CWB-VCP
1,AZU,AD,5096,2,SBCT,CWB,Afonso Pena,São José Dos Pinhais,PR,5,2017-02-02,21:28:00,2017-02-02,2017,2,2,4,FEVEREIRO,SBMG,MGF,Sílvio Name Júnior,Maringá,PR,SUL,22:27:00,2017-02-02,2017,2,2,E195,EMBRAER 195,2179.0,10964,367.0,0.98,374.09,118,107,0,946,5,1,0,0,8977,0.0,367,0,0,347182,1835,43306,39269,4023,4023,5096_2,SBCT-SBMG,CWB-MGF


In [205]:
len(flights_2017[flights_2017.sched_date.isnull()])

0

In [206]:
len(flights_2017[flights_2017.dep_date.isnull()])

0

In [207]:
flights_2017.insert(11, 'sched_year', flights_2017.sched_date.str[0:4])
flights_2017.insert(12, 'sched_month', flights_2017.sched_date.str[5:7])
flights_2017.insert(13, 'sched_day', flights_2017.sched_date.str[8:10])

In [208]:
sorted(flights_2017.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [209]:
# creating cluster column for origin airport

In [210]:
flights_2017 = flights_2017.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [211]:
len(flights_2017[flights_2017.cluster.isnull()])

0

In [212]:
flights_2017 = flights_2017.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [213]:
sorted(flights_2017.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [214]:
# creating cluster column for destination airport

In [215]:
flights_2017 = flights_2017.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [216]:
len(flights_2017[flights_2017.cluster.isnull()])

0

In [217]:
flights_2017 = flights_2017.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [218]:
sorted(flights_2017.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [220]:
# 2018 clusters – Ponta grossa (SBPG) repeated

In [221]:
clusters_2018 = pd.read_csv('./assets/csv/count/df2018.csv', sep=',')

In [222]:
clusters_2018

,icao_code,count
0,SBGR,90729
1,SBSP,80972
2,SBBR,60549
3,SBKP,44561
4,SBCF,43292
...,...,...
133,SNPY,1
134,SNFO,1
135,SNFE,1
136,SWLF,1


In [223]:
clusters_2018 = clusters_2018.merge(temp, how='left', on="icao_code")

In [224]:
clusters_2018

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,90729,-23.435556,-46.473056,Guarulhos,SP,1.0
1,SBSP,80972,-23.626111,-46.656389,São paulo,SP,2.0
2,SBBR,60549,-15.871111,-47.918611,Brasília,DF,1.0
3,SBKP,44561,-23.006944,-47.134444,Campinas,SP,3.0
4,SBCF,43292,-19.624444,-43.971944,Confins,MG,3.0
...,...,...,...,...,...,...,...
133,SNPY,1,NaN,NaN,NaN,NaN,NaN
134,SNFO,1,NaN,NaN,NaN,NaN,NaN
135,SNFE,1,NaN,NaN,NaN,NaN,NaN
136,SWLF,1,NaN,NaN,NaN,NaN,NaN


In [225]:
clusters_2018.city_primary.duplicated().any()

True

In [226]:
clusters_2018[clusters_2018.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,34978,-22.810000,-43.250556,Rio de janeiro,RJ,2.0
118,SBPG,59,-25.184444,-50.143889,Ponta grossa,PR,8.0
133,SNPY,1,NaN,NaN,NaN,NaN,NaN
134,SNFO,1,NaN,NaN,NaN,NaN,NaN
135,SNFE,1,NaN,NaN,NaN,NaN,NaN
136,SWLF,1,NaN,NaN,NaN,NaN,NaN
137,SBMN,1,NaN,NaN,NaN,NaN,NaN


In [227]:
clusters_2018[clusters_2018['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
5,SBRJ,41016,-22.91,-43.162500,Rio de janeiro,RJ,3.0
6,SBGL,34978,-22.81,-43.250556,Rio de janeiro,RJ,2.0


In [228]:
clusters_2018[clusters_2018['city_primary'] == 'Ponta grossa']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
113,SSZW,91,-25.184444,-50.143889,Ponta grossa,PR,8.0
118,SBPG,59,-25.184444,-50.143889,Ponta grossa,PR,8.0


In [229]:
clusters_2018.at[118, 'count'] = 91 + 59

In [230]:
clusters_2018.drop(113, inplace=True)

In [231]:
clusters_2018[clusters_2018['city_primary'].isnull()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
132,SNXB,1,NaN,NaN,NaN,NaN,NaN
133,SNPY,1,NaN,NaN,NaN,NaN,NaN
134,SNFO,1,NaN,NaN,NaN,NaN,NaN
135,SNFE,1,NaN,NaN,NaN,NaN,NaN
136,SWLF,1,NaN,NaN,NaN,NaN,NaN
137,SBMN,1,NaN,NaN,NaN,NaN,NaN


In [232]:
clusters_2018.dropna(inplace=True)

In [233]:
clusters_2018['cluster'] = clusters_2018['cluster'].astype(int)

In [234]:
clusters_2018 = clusters_2018.reset_index(drop=True)
clusters_2018

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,90729,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,80972,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,60549,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,44561,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,43292,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
126,SNDV,28,-20.181944,-44.870000,Divinópolis,MG,8
127,SNUH,26,-20.438889,-45.992222,Piumhi,MG,8
128,SNNU,23,-17.823056,-40.329722,Nanuque,MG,8
129,SBTC,3,-15.353333,-38.997222,Una,BA,8


In [235]:
clusters_2018.to_csv("./assets/csv/cluster/cluster_2018_map.csv", sep=",")

In [236]:
# flights 2018

In [283]:
flights_2018 = pd.read_csv("./assets/csv/temp/flights_2018.csv", sep=",", index_col=0).drop(columns="index")

In [284]:
len(flights_2018)

750379

In [285]:
len(flights_2018[flights_2018.icao_origin == flights_2018.icao_dest])

30

In [286]:
flights_2018 = flights_2018[~(flights_2018.icao_origin == flights_2018.icao_dest)]

In [287]:
flights_2018.reset_index(drop=True, inplace=True)

In [288]:
len(flights_2018)

750349

In [289]:
flights_2018.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,AZU,AD,4452,1,SBCF,CNF,Tancredo Neves,Confins,MG,4,2018-09-29,21:47:00,2018-09-29,2018,9,29,6,SETEMBRO,SBMK,MOC,Mário Ribeiro,Montes Claros,MG,SUDESTE,23:08:00,2018-09-29,2018,9,29,AT72,AEROSPATIALE/ALENIA ATR 72 FREIGHTER,900.0,6584,325.0,1.35,240.61,70,51,2,549,45,1,0,0,4570,650.0,325,0,0,178425,14625,22750,16575,2139,2139,4452_1,SBCF-SBMK,CNF-MOC
1,AZU,AD,4454,1,SBFL,FLN,Hercílio Luz,Florianópolis,SC,5,2018-09-02,16:27:00,2018-09-02,2018,9,2,7,SETEMBRO,SBPA,POA,Salgado Filho,Porto Alegre,RS,SUL,17:25:00,2018-09-02,2018,9,2,E195,EMBRAER 195,2237.0,10468,363.0,0.97,374.28,118,103,3,397,3,46,0,0,8396,1089.0,16698,0,0,144111,1089,42834,37389,3799,3799,4454_1,SBFL-SBPA,FLN-POA


In [290]:
flights_2018.loc[flights_2018.icao_origin == "SSZW", "icao_origin"] = "SBPG"

In [291]:
len(flights_2018[flights_2018.sched_date.isnull()])

0

In [292]:
len(flights_2018[flights_2018.dep_date.isnull()])

0

In [293]:
flights_2018.insert(11, 'sched_year', flights_2018.sched_date.str[0:4])
flights_2018.insert(12, 'sched_month', flights_2018.sched_date.str[5:7])
flights_2018.insert(13, 'sched_day', flights_2018.sched_date.str[8:10])

In [294]:
sorted(flights_2018.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [295]:
# creating cluster column for origin airport

In [296]:
flights_2018 = flights_2018.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [297]:
len(flights_2018[flights_2018.cluster.isnull()])

6

In [298]:
flights_2018 = flights_2018[~(flights_2018.cluster.isnull())]

In [299]:
len(flights_2018[flights_2018.cluster.isnull()])

0

In [300]:
flights_2018 = flights_2018.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [301]:
sorted(flights_2018.cluster_origin.unique())

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [302]:
flights_2018['cluster_origin'] = np.nan_to_num(flights_2018['cluster_origin']).astype(int)

In [303]:
sorted(flights_2018.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [304]:
# creating cluster column for destination airport

In [305]:
flights_2018 = flights_2018.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [306]:
len(flights_2018[flights_2018.cluster.isnull()])

6

In [307]:
flights_2018 = flights_2018[~(flights_2018.cluster.isnull())]

In [308]:
flights_2018.reset_index(drop=True, inplace=True)

In [309]:
len(flights_2018[flights_2018.cluster.isnull()])

0

In [310]:
flights_2018 = flights_2018.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [311]:
sorted(flights_2018.cluster_dest.unique())

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [312]:
flights_2018['cluster_dest'] = np.nan_to_num(flights_2018['cluster_dest']).astype(int)

In [313]:
sorted(flights_2018.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [314]:
flights_2018.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,sched_year,sched_month,sched_day,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata,lat_dd_origin,long_dd_origin,cluster_origin,lat_dd_dest,long_dd_dest,cluster_dest
0,AZU,AD,4452,1,SBCF,CNF,Tancredo Neves,Confins,MG,4,2018-09-29,2018,09,29,21:47:00,2018-09-29,2018,9,29,6,SETEMBRO,SBMK,MOC,Mário Ribeiro,Montes Claros,MG,SUDESTE,23:08:00,2018-09-29,2018,9,29,AT72,AEROSPATIALE/ALENIA ATR 72 FREIGHTER,900.0,6584,325.0,1.35,240.61,70,51,2,549,45,1,0,0,4570,650.0,325,0,0,178425,14625,22750,16575,2139,2139,4452_1,SBCF-SBMK,CNF-MOC,-19.624444,-43.971944,3,-16.706111,-43.821944,7
1,AZU,AD,4454,1,SBFL,FLN,Hercílio Luz,Florianópolis,SC,5,2018-09-02,2018,09,02,16:27:00,2018-09-02,2018,9,2,7,SETEMBRO,SBPA,POA,Salgado Filho,Porto Alegre,RS,SUL,17:25:00,2018-09-02,2018,9,2,E195,EMBRAER 195,2237.0,10468,363.0,0.97,374.28,118,103,3,397,3,46,0,0,8396,1089.0,16698,0,0,144111,1089,42834,37389,3799,3799,4454_1,SBFL-SBPA,FLN-POA,-27.670278,-48.552500,4,-29.994722,-51.171111,3


In [315]:
flights_2018.to_csv('./assets/csv/flights_2018.csv', sep=',')

In [316]:
# 2019 clusters – Vitória da conquista (SBVC), Arealva (SBPG), Sinop (SBPG) and Ponta grossa (SBPG) repeated

In [317]:
clusters_2019 = pd.read_csv('./assets/csv/count/df2019.csv', sep=',')

In [318]:
clusters_2019

,icao_code,count
0,SBGR,94058
1,SBSP,84449
2,SBBR,56104
3,SBKP,46569
4,SBCF,44850
...,...,...
155,SNAR,20
156,SBPC,15
157,SBAQ,8
158,SNUH,7


In [319]:
clusters_2019 = clusters_2019.merge(temp, how='left', on="icao_code")

In [320]:
clusters_2019

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,94058,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,84449,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,56104,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,46569,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,44850,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
155,SNAR,20,-16.183889,-40.667222,Almenara,MG,8
156,SBPC,15,-21.837778,-46.566111,Poços de caldas,MG,8
157,SBAQ,8,-21.804444,-48.140278,Araraquara,SP,8
158,SNUH,7,-20.438889,-45.992222,Piumhi,MG,8


In [321]:
clusters_2019.city_primary.duplicated().any()

True

In [322]:
clusters_2019[clusters_2019.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBGL,31921,-22.810000,-43.250556,Rio de janeiro,RJ,2
50,SBVC,1160,-14.907778,-40.914722,Vitória da conquista,BA,7
110,SNYA,120,-1.479167,-52.578056,Almeirim,PA,8
112,SBJR,118,-22.987500,-43.370000,Rio de janeiro,RJ,8
115,SJTC,111,-22.157778,-49.068333,Arealva,SP,8
131,SBSI,56,-11.885000,-55.586111,Sinop,MT,7
138,SSZW,48,-25.184444,-50.143889,Ponta grossa,PR,8


In [323]:
clusters_2019[clusters_2019['city_primary'].isnull()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster


In [324]:
clusters_2019[clusters_2019['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
5,SBRJ,40299,-22.9100,-43.162500,Rio de janeiro,RJ,3
6,SBGL,31921,-22.8100,-43.250556,Rio de janeiro,RJ,2
112,SBJR,118,-22.9875,-43.370000,Rio de janeiro,RJ,8


In [325]:
clusters_2019[clusters_2019['city_primary'] == 'Vitória da conquista']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
49,SBQV,1221,-14.907778,-40.914722,Vitória da conquista,BA,7
50,SBVC,1160,-14.907778,-40.914722,Vitória da conquista,BA,7


In [326]:
clusters_2019.at[50, 'count'] = 1221 + 1160

In [327]:
clusters_2019.drop(49, inplace=True)

In [328]:
clusters_2019[clusters_2019['city_primary'] == 'Almeirim']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
109,SBMD,120,-0.889722,-52.602222,Almeirim,PA,8
110,SNYA,120,-1.479167,-52.578056,Almeirim,PA,8


In [329]:
clusters_2019[clusters_2019['city_primary'] == 'Arealva']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
53,SBAE,991,-22.157778,-49.068333,Arealva,SP,8
115,SJTC,111,-22.157778,-49.068333,Arealva,SP,8


In [330]:
clusters_2019.at[53, 'count'] = 991 + 111

In [331]:
clusters_2019.drop(115, inplace=True)

In [332]:
clusters_2019[clusters_2019['city_primary'] == 'Sinop']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
61,SWSI,784,-11.885,-55.586111,Sinop,MT,7
131,SBSI,56,-11.885,-55.586111,Sinop,MT,7


In [333]:
clusters_2019.at[131, 'count'] = 784 + 56

In [334]:
clusters_2019.drop(61, inplace=True)

In [335]:
clusters_2019[clusters_2019['city_primary'] == 'Ponta grossa']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
105,SBPG,128,-25.184444,-50.143889,Ponta grossa,PR,8
138,SSZW,48,-25.184444,-50.143889,Ponta grossa,PR,8


In [336]:
clusters_2019.at[105, 'count'] = 128 + 48

In [337]:
clusters_2019.drop(48, inplace=True)

In [338]:
clusters_2019 = clusters_2019.reset_index(drop=True)
clusters_2019

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,94058,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,84449,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,56104,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,46569,-23.006944,-47.134444,Campinas,SP,3
4,SBCF,44850,-19.624444,-43.971944,Confins,MG,3
...,...,...,...,...,...,...,...
151,SNAR,20,-16.183889,-40.667222,Almenara,MG,8
152,SBPC,15,-21.837778,-46.566111,Poços de caldas,MG,8
153,SBAQ,8,-21.804444,-48.140278,Araraquara,SP,8
154,SNUH,7,-20.438889,-45.992222,Piumhi,MG,8


In [339]:
clusters_2019.to_csv("./assets/csv/cluster/cluster_2019_map.csv", sep=",")

In [340]:
# flights 2019

In [341]:
flights_2019 = pd.read_csv("./assets/csv/temp/flights_2019.csv", sep=",", index_col=0).drop(columns="index")

In [342]:
len(flights_2019)

754888

In [343]:
len(flights_2019[flights_2019.icao_origin == flights_2019.icao_dest])

22

In [344]:
flights_2019 = flights_2019[~(flights_2019.icao_origin == flights_2019.icao_dest)]

In [345]:
flights_2019.reset_index(drop=True, inplace=True)

In [346]:
len(flights_2019)

754866

In [347]:
flights_2019.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,GLO,G3,1594,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2019-07-01,08:04:00,2019-07-01,2019,7,1,1,JULHO,SBBR,BSB,Presidente Juscelino Kubitschek,Brasília,DF,CENTRO-OESTE,09:36:00,2019-07-01,2019,7,1,B738,BOEING 737-800 (WINGLETS) PAX,4148.0,19800,873.0,1.53,570.33,186,177,1,835,0,1621,0,80,15886,873.0,1415130,69840,0,728955,0,162378,154521,17285,17285,1594_1,SBSP-SBBR,CGH-BSB
1,GLO,G3,2007,1,SBBE,BEL,Internacional De Belém/Val De Cans/Júlio Cezar...,Belém,PA,1,2019-07-01,02:57:00,2019-07-01,2019,7,1,1,JULHO,SBGL,GIG,Aeroporto Internacional Do Rio De Janeiro (Gal...,Rio De Janeiro,RJ,SUDESTE,06:30:00,2019-07-01,2019,7,1,B738,BOEING 737-800 (WINGLETS) PAX,10789.0,19800,2448.0,3.55,689.69,186,178,5,2035,0,2,0,0,15762,12240.0,4896,0,0,4981680,0,455328,435744,48470,48470,2007_1,SBBE-SBGL,BEL-GIG


In [348]:
flights_2019.loc[flights_2019.icao_origin == "SSZW", "icao_origin"] = "SBPG"

In [349]:
flights_2019.loc[flights_2019.icao_origin == "SWSI", "icao_origin"] = "SBSI"

In [350]:
flights_2019.loc[flights_2019.icao_origin == "SJTC", "icao_origin"] = "SBAE"

In [351]:
len(flights_2019[flights_2019.sched_date.isnull()])

0

In [352]:
len(flights_2019[flights_2019.dep_date.isnull()])

0

In [353]:
flights_2019.insert(11, 'sched_year', flights_2019.sched_date.str[0:4])
flights_2019.insert(12, 'sched_month', flights_2019.sched_date.str[5:7])
flights_2019.insert(13, 'sched_day', flights_2019.sched_date.str[8:10])

In [354]:
sorted(flights_2019.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [355]:
# creating cluster column for origin airport

In [356]:
flights_2019 = flights_2019.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [357]:
len(flights_2019[flights_2019.cluster.isnull()])

0

In [358]:
flights_2019 = flights_2019.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [359]:
sorted(flights_2019.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [360]:
# creating cluster column for destination airport

In [361]:
flights_2019 = flights_2019.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [362]:
len(flights_2019[flights_2019.cluster.isnull()])

0

In [363]:
flights_2019 = flights_2019.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [364]:
sorted(flights_2019.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [367]:
flights_2019.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,sched_year,sched_month,sched_day,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata,lat_dd_origin,long_dd_origin,cluster_origin,lat_dd_dest,long_dd_dest,cluster_dest
0,GLO,G3,1594,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2019-07-01,2019,07,01,08:04:00,2019-07-01,2019,7,1,1,JULHO,SBBR,BSB,Presidente Juscelino Kubitschek,Brasília,DF,CENTRO-OESTE,09:36:00,2019-07-01,2019,7,1,B738,BOEING 737-800 (WINGLETS) PAX,4148.0,19800,873.0,1.53,570.33,186,177,1,835,0,1621,0,80,15886,873.0,1415130,69840,0,728955,0,162378,154521,17285,17285,1594_1,SBSP-SBBR,CGH-BSB,-23.626111,-46.656389,2,-15.871111,-47.918611,1
1,GLO,G3,2007,1,SBBE,BEL,Internacional De Belém/Val De Cans/Júlio Cezar...,Belém,PA,1,2019-07-01,2019,07,01,02:57:00,2019-07-01,2019,7,1,1,JULHO,SBGL,GIG,Aeroporto Internacional Do Rio De Janeiro (Gal...,Rio De Janeiro,RJ,SUDESTE,06:30:00,2019-07-01,2019,7,1,B738,BOEING 737-800 (WINGLETS) PAX,10789.0,19800,2448.0,3.55,689.69,186,178,5,2035,0,2,0,0,15762,12240.0,4896,0,0,4981680,0,455328,435744,48470,48470,2007_1,SBBE-SBGL,BEL-GIG,-1.384722,-48.478889,3,-22.810000,-43.250556,2


In [369]:
# 2020 clusters – Vitória da conquista (SBVC), Arealva (SBPG) and Sinop (SBSI) repeated

In [370]:
clusters_2020 = pd.read_csv('./assets/csv/count/df2020.csv', sep=',')

In [371]:
clusters_2020

,icao_code,count
0,SBGR,53941
1,SBSP,28312
2,SBBR,28175
3,SBKP,27906
4,SBRJ,22083
...,...,...
171,SIZX,4
172,SILC,4
173,SBBH,2
174,SBGS,1


In [372]:
clusters_2020 = clusters_2020.merge(temp, how='left', on="icao_code")

In [373]:
clusters_2020

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,53941,-23.435556,-46.473056,Guarulhos,SP,1.0
1,SBSP,28312,-23.626111,-46.656389,São paulo,SP,2.0
2,SBBR,28175,-15.871111,-47.918611,Brasília,DF,1.0
3,SBKP,27906,-23.006944,-47.134444,Campinas,SP,3.0
4,SBRJ,22083,-22.910000,-43.162500,Rio de janeiro,RJ,3.0
...,...,...,...,...,...,...,...
171,SIZX,4,-11.296667,-57.548889,Juara,MT,8.0
172,SILC,4,-13.037778,-55.950278,Lucas do rio verde,MT,8.0
173,SBBH,2,-19.851944,-43.950556,Belo horizonte,MG,6.0
174,SBGS,1,NaN,NaN,NaN,NaN,NaN


In [374]:
clusters_2020[clusters_2020['city_primary'].isnull()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
174,SBGS,1,NaN,NaN,NaN,NaN,NaN
175,SBMT,1,NaN,NaN,NaN,NaN,NaN


In [375]:
clusters_2020.dropna(inplace=True)

In [376]:
clusters_2020['cluster'] = clusters_2020['cluster'].astype(int)

In [377]:
clusters_2020.city_primary.duplicated().any()

True

In [378]:
clusters_2020[clusters_2020.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
9,SBGL,11008,-22.810000,-43.250556,Rio de janeiro,RJ,2
73,SWSI,233,-11.885000,-55.586111,Sinop,MT,7
85,SBJR,150,-22.987500,-43.370000,Rio de janeiro,RJ,8
87,SBQV,142,-14.907778,-40.914722,Vitória da conquista,BA,7
90,SNYA,131,-1.479167,-52.578056,Almeirim,PA,8
112,SJTC,68,-22.157778,-49.068333,Arealva,SP,8
144,SBTB,30,-1.489722,-56.396944,Oriximiná,PA,8
158,SBAC,13,-4.568611,-37.804722,Aracati,CE,8
164,SBJC,11,-1.412902,-48.463912,Belém,PA,8


In [379]:
clusters_2020[clusters_2020['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
4,SBRJ,22083,-22.9100,-43.162500,Rio de janeiro,RJ,3
9,SBGL,11008,-22.8100,-43.250556,Rio de janeiro,RJ,2
85,SBJR,150,-22.9875,-43.370000,Rio de janeiro,RJ,8


In [380]:
clusters_2020[clusters_2020['city_primary'] == 'Almeirim']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
80,SBMD,180,-0.889722,-52.602222,Almeirim,PA,8
90,SNYA,131,-1.479167,-52.578056,Almeirim,PA,8


In [381]:
clusters_2020[clusters_2020['city_primary'] == 'Arealva']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
46,SBAE,760,-22.157778,-49.068333,Arealva,SP,8
112,SJTC,68,-22.157778,-49.068333,Arealva,SP,8


In [382]:
clusters_2020.at[46, 'count'] = 760 + 68

In [383]:
clusters_2020.drop(112, inplace=True)

In [384]:
clusters_2020[clusters_2020['city_primary'] == 'Sinop']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
57,SBSI,418,-11.885,-55.586111,Sinop,MT,7
73,SWSI,233,-11.885,-55.586111,Sinop,MT,7


In [385]:
clusters_2020.at[57, 'count'] = 418 + 233

In [386]:
clusters_2020.drop(73, inplace=True)

In [387]:
clusters_2020[clusters_2020['city_primary'] == 'Vitória da conquista']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
40,SBVC,1013,-14.907778,-40.914722,Vitória da conquista,BA,7
87,SBQV,142,-14.907778,-40.914722,Vitória da conquista,BA,7


In [388]:
clusters_2020.at[40, 'count'] = 1013 + 142

In [389]:
clusters_2020.drop(87, inplace=True)

In [390]:
clusters_2020[clusters_2020['city_primary'] == 'Oriximiná']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
102,SNOX,87,-1.713611,-55.835833,Oriximiná,PA,8
144,SBTB,30,-1.489722,-56.396944,Oriximiná,PA,8


In [391]:
clusters_2020[clusters_2020['city_primary'] == 'Aracati']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
137,SNAT,35,-4.568611,-37.804722,Aracati,CE,8
158,SBAC,13,-4.568611,-37.804722,Aracati,CE,8


In [392]:
clusters_2020[clusters_2020['city_primary'] == 'Belém']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
12,SBBE,8003,-1.384722,-48.478889,Belém,PA,3
164,SBJC,11,-1.412902,-48.463912,Belém,PA,8


In [393]:
clusters_2020 = clusters_2020.reset_index(drop=True)
clusters_2020

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,53941,-23.435556,-46.473056,Guarulhos,SP,1
1,SBSP,28312,-23.626111,-46.656389,São paulo,SP,2
2,SBBR,28175,-15.871111,-47.918611,Brasília,DF,1
3,SBKP,27906,-23.006944,-47.134444,Campinas,SP,3
4,SBRJ,22083,-22.910000,-43.162500,Rio de janeiro,RJ,3
...,...,...,...,...,...,...,...
166,SWBE,8,-4.042778,-40.893889,São benedito,CE,8
167,SDZG,8,-5.933333,-40.297500,Tauá,CE,8
168,SIZX,4,-11.296667,-57.548889,Juara,MT,8
169,SILC,4,-13.037778,-55.950278,Lucas do rio verde,MT,8


In [ ]:
# flights 2020

In [467]:
flights_2020 = pd.read_csv("./assets/csv/temp/flights_2020.csv", sep=",", index_col=0).drop(columns="index")

In [468]:
len(flights_2020)

366365

In [469]:
len(flights_2020[flights_2020.icao_origin == flights_2020.icao_dest])

21

In [470]:
flights_2020 = flights_2020[~(flights_2020.icao_origin == flights_2020.icao_dest)]

In [471]:
flights_2020.reset_index(drop=True, inplace=True)

In [472]:
len(flights_2020)

366344

In [473]:
flights_2020.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,PTB,2Z,2224,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2020-08-17,08:43:00,2020-08-17,2020,8,17,1,AGOSTO,SBRJ,SDU,Santos Dumont,Rio De Janeiro,RJ,SUDESTE,09:58:00,2020-08-17,2020,8,17,AT76,AEROSPATIALE/ALENIA ATR 72-600/72-212A (600),4788,7237,366,1.25,292.50,70,57.0,2.0,200,0,0.0,0.0,0.0,4625.0,732.0,0,0,0,73200,0,25620,20862,2648,2648,2224_1,SBSP-SBRJ,CGH-SDU
1,PTB,2Z,2222,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2020-08-06,20:00:00,2020-08-06,2020,8,6,4,AGOSTO,SBRP,RAO,Leite Lopes,Ribeirão Preto,SP,SUDESTE,21:09:00,2020-08-06,2020,8,6,AT76,AEROSPATIALE/ALENIA ATR 72-600/72-212A (600),850,7237,300,1.15,260.74,70,27.0,5.0,111,0,0.0,0.0,0.0,2511.0,1500.0,0,0,0,33300,0,21000,8100,2171,2171,2222_1,SBSP-SBRP,CGH-RAO


In [474]:
flights_2020.loc[flights_2020.icao_origin == "SBQV", "icao_origin"] = "SBVC"

In [475]:
flights_2020.loc[flights_2020.icao_origin == "SWSI", "icao_origin"] = "SBSI"

In [476]:
flights_2020.loc[flights_2020.icao_origin == "SJTC", "icao_origin"] = "SBAE"

In [477]:
len(flights_2020[flights_2020.sched_date.isnull()])

0

In [478]:
len(flights_2020[flights_2020.dep_date.isnull()])

0

In [479]:
flights_2020.insert(11, 'sched_year', flights_2020.sched_date.str[0:4])
flights_2020.insert(12, 'sched_month', flights_2020.sched_date.str[5:7])
flights_2020.insert(13, 'sched_day', flights_2020.sched_date.str[8:10])

In [480]:
sorted(flights_2020.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [481]:
# creating cluster column for origin airport

In [482]:
flights_2020 = flights_2020.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [483]:
len(flights_2020[flights_2020.cluster.isnull()])

2

In [484]:
flights_2020 = flights_2020[~(flights_2020.cluster.isnull())]

In [485]:
flights_2020.reset_index(drop=True, inplace=True)

In [486]:
len(flights_2020[flights_2020.cluster.isnull()])

0

In [487]:
flights_2020 = flights_2020.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [488]:
sorted(flights_2020.cluster_origin.unique())

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [489]:
flights_2020['cluster_origin'] = np.nan_to_num(flights_2020['cluster_origin']).astype(int)

In [490]:
sorted(flights_2020.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [491]:
# creating cluster column for destination airport

In [492]:
flights_2020 = flights_2020.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [493]:
len(flights_2020[flights_2020.cluster.isnull()])

3

In [494]:
flights_2020 = flights_2020[~(flights_2020.cluster.isnull())]

In [495]:
flights_2020.reset_index(drop=True, inplace=True)

In [496]:
len(flights_2020[flights_2020.cluster.isnull()])

0

In [497]:
flights_2020 = flights_2020.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [498]:
sorted(flights_2020.cluster_dest.unique())

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [499]:
flights_2020['cluster_dest'] = np.nan_to_num(flights_2020['cluster_dest']).astype(int)

In [500]:
sorted(flights_2020.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [501]:
flights_2020.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,sched_year,sched_month,sched_day,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata,lat_dd_origin,long_dd_origin,cluster_origin,lat_dd_dest,long_dd_dest,cluster_dest
0,PTB,2Z,2224,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2020-08-17,2020,08,17,08:43:00,2020-08-17,2020,8,17,1,AGOSTO,SBRJ,SDU,Santos Dumont,Rio De Janeiro,RJ,SUDESTE,09:58:00,2020-08-17,2020,8,17,AT76,AEROSPATIALE/ALENIA ATR 72-600/72-212A (600),4788,7237,366,1.25,292.50,70,57.0,2.0,200,0,0.0,0.0,0.0,4625.0,732.0,0,0,0,73200,0,25620,20862,2648,2648,2224_1,SBSP-SBRJ,CGH-SDU,-23.626111,-46.656389,2,-22.910000,-43.162500,3
1,PTB,2Z,2222,1,SBSP,CGH,Congonhas,São Paulo,SP,4,2020-08-06,2020,08,06,20:00:00,2020-08-06,2020,8,6,4,AGOSTO,SBRP,RAO,Leite Lopes,Ribeirão Preto,SP,SUDESTE,21:09:00,2020-08-06,2020,8,6,AT76,AEROSPATIALE/ALENIA ATR 72-600/72-212A (600),850,7237,300,1.15,260.74,70,27.0,5.0,111,0,0.0,0.0,0.0,2511.0,1500.0,0,0,0,33300,0,21000,8100,2171,2171,2222_1,SBSP-SBRP,CGH-RAO,-23.626111,-46.656389,2,-21.136389,-47.776667,6


In [504]:
# 2021 clusters – Vitória da conquista (SBVC) repeated

In [505]:
clusters_2021 = pd.read_csv('./assets/csv/count/df2021.csv', sep=',')

In [506]:
clusters_2021

,icao_code,count
0,SBGR,40500
1,SBKP,27677
2,SBBR,21654
3,SBRF,19076
4,SBSP,18961
...,...,...
134,SSSC,6
135,SSLT,5
136,SBPG,2
137,SBUY,1


In [507]:
clusters_2021 = clusters_2021.merge(temp, how='left', on="icao_code")

In [508]:
clusters_2021

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,40500,-23.435556,-46.473056,Guarulhos,SP,1
1,SBKP,27677,-23.006944,-47.134444,Campinas,SP,3
2,SBBR,21654,-15.871111,-47.918611,Brasília,DF,1
3,SBRF,19076,-8.126389,-34.922778,Recife,PE,2
4,SBSP,18961,-23.626111,-46.656389,São paulo,SP,2
...,...,...,...,...,...,...,...
134,SSSC,6,-29.684167,-52.412222,Santa cruz do sul,RS,8
135,SSLT,5,-29.812500,-55.893333,Alegrete,RS,8
136,SBPG,2,-25.184444,-50.143889,Ponta grossa,PR,8
137,SBUY,1,-4.883056,-65.355833,Coari,AM,8


In [509]:
clusters_2021.city_primary.duplicated().any()

True

In [510]:
clusters_2021[clusters_2021['city_primary'].isnull()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster


In [511]:
clusters_2021[clusters_2021.city_primary.duplicated()]

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
12,SBGL,5582,-22.810000,-43.250556,Rio de janeiro,RJ,2
68,SBQV,214,-14.907778,-40.914722,Vitória da conquista,BA,7
98,SNYA,93,-1.479167,-52.578056,Almeirim,PA,8
137,SBUY,1,-4.883056,-65.355833,Coari,AM,8


In [512]:
clusters_2021[clusters_2021['city_primary'] == 'Rio de janeiro']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
6,SBRJ,16761,-22.91,-43.162500,Rio de janeiro,RJ,3
12,SBGL,5582,-22.81,-43.250556,Rio de janeiro,RJ,2


In [513]:
clusters_2021[clusters_2021['city_primary'] == 'Almeirim']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
73,SBMD,171,-0.889722,-52.602222,Almeirim,PA,8
98,SNYA,93,-1.479167,-52.578056,Almeirim,PA,8


In [514]:
clusters_2021[clusters_2021['city_primary'] == 'Vitória da conquista']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
45,SBVC,671,-14.907778,-40.914722,Vitória da conquista,BA,7
68,SBQV,214,-14.907778,-40.914722,Vitória da conquista,BA,7


In [515]:
clusters_2021.at[45, 'count'] = 671 + 214

In [516]:
clusters_2021.drop(68, inplace=True)

In [517]:
clusters_2021[clusters_2021['city_primary'] == 'Coari']

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
76,SWKO,163,-4.133889,-63.131111,Coari,AM,8
137,SBUY,1,-4.883056,-65.355833,Coari,AM,8


In [518]:
clusters_2021 = clusters_2021.reset_index(drop=True)
clusters_2021

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster
0,SBGR,40500,-23.435556,-46.473056,Guarulhos,SP,1
1,SBKP,27677,-23.006944,-47.134444,Campinas,SP,3
2,SBBR,21654,-15.871111,-47.918611,Brasília,DF,1
3,SBRF,19076,-8.126389,-34.922778,Recife,PE,2
4,SBSP,18961,-23.626111,-46.656389,São paulo,SP,2
...,...,...,...,...,...,...,...
133,SSSC,6,-29.684167,-52.412222,Santa cruz do sul,RS,8
134,SSLT,5,-29.812500,-55.893333,Alegrete,RS,8
135,SBPG,2,-25.184444,-50.143889,Ponta grossa,PR,8
136,SBUY,1,-4.883056,-65.355833,Coari,AM,8


In [519]:
# flights 2021

In [520]:
flights_2021 = pd.read_csv("./assets/csv/temp/flights_2021.csv", sep=",", index_col=0).drop(columns="index")

In [522]:
len(flights_2021)

295023

In [523]:
len(flights_2021[flights_2021.icao_origin == flights_2021.icao_dest])

6

In [524]:
flights_2021 = flights_2021[~(flights_2021.icao_origin == flights_2021.icao_dest)]

In [525]:
flights_2021.reset_index(drop=True, inplace=True)

In [526]:
len(flights_2021)

295017

In [527]:
flights_2021.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata
0,PAM,7M,5964,1,SBEG,MAO,Eduardo Gomes,Manaus,AM,1,2021-03-05,19:05:00,2021-03-05,2021,3,5,5,MARÇO,SWPI,PIN,Júlio Bélem,Parintins,AM,NORTE,20:17:00,2021-03-05,2021,3,5,AT45,AEROSPATIALE/ALENIA ATR 42-500,763,4433,367,1.2,305.45,47,24,0,166,0,0,0,0,1966,0,0,0,0,60922,0,17249,8808,1626,1626,5964_1,SBEG-SWPI,MAO-PIN
1,PAM,7M,5934,1,SBEG,MAO,Eduardo Gomes,Manaus,AM,1,2021-03-18,08:00:00,2021-03-18,2021,3,18,4,MARÇO,SBTF,TFF,Tefé,Tefé,AM,NORTE,09:30:00,2021-03-18,2021,3,18,AT75,AEROSPATIALE/ALENIA ATR 72-500/72-212A (500),1159,7000,520,1.5,346.91,72,44,3,567,0,0,0,0,4092,1560,0,0,0,294840,0,37440,22880,3640,3640,5934_1,SBEG-SBTF,MAO-TFF


In [528]:
flights_2021.loc[flights_2021.icao_origin == "SBQV", "icao_origin"] = "SBVC"

In [529]:
len(flights_2021[flights_2021.sched_date.isnull()])

0

In [530]:
len(flights_2021[flights_2021.dep_date.isnull()])

0

In [531]:
flights_2021.insert(11, 'sched_year', flights_2021.sched_date.str[0:4])
flights_2021.insert(12, 'sched_month', flights_2021.sched_date.str[5:7])
flights_2021.insert(13, 'sched_day', flights_2021.sched_date.str[8:10])

In [532]:
sorted(flights_2021.dep_month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 12]

In [533]:
# creating cluster column for origin airport

In [534]:
flights_2021 = flights_2021.merge(
    temp, how="left", left_on="icao_origin", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [535]:
len(flights_2021[flights_2021.cluster.isnull()])

0

In [536]:
flights_2021 = flights_2021.rename(
    {
        "lat_dd": "lat_dd_origin",
        "long_dd": "long_dd_origin",
        "cluster": "cluster_origin"
    },
    axis=1,
)

In [537]:
sorted(flights_2021.cluster_origin.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [538]:
# creating cluster column for destination airport

In [539]:
flights_2021 = flights_2021.merge(
    temp, how="left", left_on="icao_dest", right_on="icao_code"
).drop(columns=["icao_code", "city_primary", "state"])

In [540]:
len(flights_2021[flights_2021.cluster.isnull()])

0

In [541]:
flights_2021 = flights_2021.rename(
    {
        "lat_dd": "lat_dd_dest",
        "long_dd": "long_dd_dest",
        "cluster": "cluster_dest"
    },
    axis=1,
)

In [542]:
sorted(flights_2021.cluster_dest.unique())

[1, 2, 3, 4, 5, 6, 7, 8]

In [544]:
flights_2021.head(2)

,icao_carrier,iata_carrier,flight,flight_step,icao_origin,iata_origin,dep_airport_name,dep_city,dep_state,dep_region,sched_date,sched_year,sched_month,sched_day,dep_time,dep_date,dep_year,dep_month,dep_day,dep_week_day,reference_month,icao_dest,iata_dest,arr_airport_name,arr_city,arr_state,arr_region,arr_time,arr_date,arr_year,arr_month,arr_day,icao_aircraft_type,aircraft_model,fuel_consumption_l,payload_kg,flight_distance_km,flight_hrs,flight_speed_avg,seats_available,seats_sold,seats_free,pax_baggage_free_kg,pax_baggage_paid_kg,cargo_paid_kg,cargo_mail_kg,cargo_free_kg,aircraft_freight_kg,pax_free_km,cargo_paid_km,cargo_free_km,cargo_mail_km,baggage_free_km,baggage_paid_km,available_seat_km,revenue_pax_km,available_tonne_km,revenue_tonne_km,flight_unique_nr,route_icao,route_iata,lat_dd_origin,long_dd_origin,cluster_origin,lat_dd_dest,long_dd_dest,cluster_dest
0,PAM,7M,5964,1,SBEG,MAO,Eduardo Gomes,Manaus,AM,1,2021-03-05,2021,03,05,19:05:00,2021-03-05,2021,3,5,5,MARÇO,SWPI,PIN,Júlio Bélem,Parintins,AM,NORTE,20:17:00,2021-03-05,2021,3,5,AT45,AEROSPATIALE/ALENIA ATR 42-500,763,4433,367,1.2,305.45,47,24,0,166,0,0,0,0,1966,0,0,0,0,60922,0,17249,8808,1626,1626,5964_1,SBEG-SWPI,MAO-PIN,-3.041111,-60.050556,3,-2.669444,-56.771111,8
1,PAM,7M,5934,1,SBEG,MAO,Eduardo Gomes,Manaus,AM,1,2021-03-18,2021,03,18,08:00:00,2021-03-18,2021,3,18,4,MARÇO,SBTF,TFF,Tefé,Tefé,AM,NORTE,09:30:00,2021-03-18,2021,3,18,AT75,AEROSPATIALE/ALENIA ATR 72-500/72-212A (500),1159,7000,520,1.5,346.91,72,44,3,567,0,0,0,0,4092,1560,0,0,0,294840,0,37440,22880,3640,3640,5934_1,SBEG-SBTF,MAO-TFF,-3.041111,-60.050556,3,-3.380278,-64.725278,8


In [1]:
# loading libraries

import glob
import os

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from pywaffle import Waffle

- Function for general info by YEAR/MONTH/DAY

In [3]:
files_list = [x for x in os.listdir("./assets/csv/cluster/") if x.endswith("_map.csv")]
files_list.sort()

In [10]:
def concat_clusters(file):
    file_year = file[8:12]
    df = pd.read_csv(f"./assets/csv/cluster/{file}", sep=",").drop(columns='Unnamed: 0')
    df["year"] = file_year
    return df

In [11]:
df_list = []

for file in files_list:
    print(file)
    df_list.append(concat_clusters(file))

df_clusters_all_map = pd.concat(df_list)

cluster_2002_map.csv
cluster_2003_map.csv
cluster_2004_map.csv
cluster_2005_map.csv
cluster_2006_map.csv
cluster_2007_map.csv
cluster_2008_map.csv
cluster_2009_map.csv
cluster_2010_map.csv
cluster_2011_map.csv
cluster_2012_map.csv
cluster_2013_map.csv
cluster_2014_map.csv
cluster_2015_map.csv
cluster_2016_map.csv
cluster_2017_map.csv
cluster_2018_map.csv
cluster_2019_map.csv
cluster_2020_map.csv
cluster_2021_map.csv


In [12]:
df_clusters_all_map.to_csv("./assets/csv/cluster/df_clusters_all_map.csv", sep=',')

In [13]:
df_clusters_all_map

,icao_code,count,lat_dd,long_dd,city_primary,state,cluster,year
0,SBBR,7972,-15.871111,-47.918611,Brasília,DF,1,2002
1,SBSP,7647,-23.626111,-46.656389,São paulo,SP,2,2002
2,SBGR,6722,-23.435556,-46.473056,Guarulhos,SP,1,2002
3,SBSV,5727,-12.908611,-38.322500,Salvador,BA,2,2002
4,SBGL,5189,-22.810000,-43.250556,Rio de janeiro,RJ,2,2002
...,...,...,...,...,...,...,...,...
133,SSSC,6,-29.684167,-52.412222,Santa cruz do sul,RS,8,2021
134,SSLT,5,-29.812500,-55.893333,Alegrete,RS,8,2021
135,SBPG,2,-25.184444,-50.143889,Ponta grossa,PR,8,2021
136,SBUY,1,-4.883056,-65.355833,Coari,AM,8,2021
